<a href="https://colab.research.google.com/github/ThisIsFarhan/FlanT5-FineTuning/blob/main/T5_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82

In [2]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [3]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-small"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
# Acquire the training data from Hugging Face
DATA_NAME = "yahoo_answers_qa"
yahoo_answers_qa = load_dataset(DATA_NAME)

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

yahoo_answers_qa.py:   0%|          | 0.00/2.76k [00:00<?, ?B/s]

The repository for yahoo_answers_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/yahoo_answers_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/87362 [00:00<?, ? examples/s]

In [12]:
yahoo_answers_qa = yahoo_answers_qa["train"].train_test_split(test_size=0.3)

In [14]:
yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 61153
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 26209
    })
})

In [15]:
yahoo_answers_qa["train"] = yahoo_answers_qa["train"].select(range(1000))
yahoo_answers_qa["test"] = yahoo_answers_qa["test"].select(range(1000))

In [16]:
yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 1000
    })
})

In [19]:
prefix = "Please answer this question: "
inputs = [prefix + doc for doc in yahoo_answers_qa["train"]["question"]]
model_inputs = tokenizer(inputs, max_length=128, truncation=True)

In [20]:
inputs

['Please answer this question: Why do Catholics have to go to Priest to confess and recieve a punishment like saying "Hail Mary" a bunch of-',
 'Please answer this question: What do red and blue mean in my aura?',
 'Please answer this question: how to become a don ?',
 'Please answer this question: how do Appalachians eat their grits what sides and how are they cooked?',
 'Please answer this question: How do I grill the perfect steak on a gas grill?',
 'Please answer this question: how can i get schoolrship?',
 'Please answer this question: ok, how do you clean  the laser lens on a PS2?',
 'Please answer this question: how to delete  pics from apple ipod?',
 'Please answer this question: What is the meaning of life?',
 'Please answer this question: how do the measurements on tires work?',
 'Please answer this question: Describe in short human WBC & RBC?',
 'Please answer this question: why are all of my programs working slow?',
 "Please answer this question: What is the DNA sequence, f

In [21]:
model_inputs

{'input_ids': [[863, 1525, 48, 822, 10, 1615, 103, 6502, 7, 43, 12, 281, 12, 24582, 12, 25613, 11, 3, 7886, 15, 162, 3, 9, 19372, 114, 2145, 96, 566, 9, 173, 3790, 121, 3, 9, 7292, 13, 18, 1], [863, 1525, 48, 822, 10, 363, 103, 1131, 11, 1692, 1243, 16, 82, 9437, 58, 1], [863, 1525, 48, 822, 10, 149, 12, 582, 3, 9, 278, 3, 58, 1], [863, 1525, 48, 822, 10, 149, 103, 2276, 138, 1836, 7137, 3, 1544, 70, 3, 3496, 17, 7, 125, 4458, 11, 149, 33, 79, 8311, 58, 1], [863, 1525, 48, 822, 10, 571, 103, 27, 6903, 8, 626, 17718, 30, 3, 9, 1807, 6903, 58, 1], [863, 1525, 48, 822, 10, 149, 54, 3, 23, 129, 496, 52, 2009, 58, 1], [863, 1525, 48, 822, 10, 3, 1825, 6, 149, 103, 25, 1349, 8, 6124, 6888, 30, 3, 9, 5610, 357, 58, 1], [863, 1525, 48, 822, 10, 149, 12, 9268, 14352, 45, 8947, 3, 23, 11410, 58, 1], [863, 1525, 48, 822, 10, 363, 19, 8, 2530, 13, 280, 58, 1], [863, 1525, 48, 822, 10, 149, 103, 8, 11110, 30, 12541, 161, 58, 1], [863, 1525, 48, 822, 10, 3, 30010, 16, 710, 936, 549, 7645, 3, 184, 39

In [22]:
labels = tokenizer(text_target=yahoo_answers_qa["train"]["answer"],
                      max_length=512,
                      truncation=True)

In [23]:
model_inputs["labels"] = labels["input_ids"]

In [24]:
model_inputs

{'input_ids': [[863, 1525, 48, 822, 10, 1615, 103, 6502, 7, 43, 12, 281, 12, 24582, 12, 25613, 11, 3, 7886, 15, 162, 3, 9, 19372, 114, 2145, 96, 566, 9, 173, 3790, 121, 3, 9, 7292, 13, 18, 1], [863, 1525, 48, 822, 10, 363, 103, 1131, 11, 1692, 1243, 16, 82, 9437, 58, 1], [863, 1525, 48, 822, 10, 149, 12, 582, 3, 9, 278, 3, 58, 1], [863, 1525, 48, 822, 10, 149, 103, 2276, 138, 1836, 7137, 3, 1544, 70, 3, 3496, 17, 7, 125, 4458, 11, 149, 33, 79, 8311, 58, 1], [863, 1525, 48, 822, 10, 571, 103, 27, 6903, 8, 626, 17718, 30, 3, 9, 1807, 6903, 58, 1], [863, 1525, 48, 822, 10, 149, 54, 3, 23, 129, 496, 52, 2009, 58, 1], [863, 1525, 48, 822, 10, 3, 1825, 6, 149, 103, 25, 1349, 8, 6124, 6888, 30, 3, 9, 5610, 357, 58, 1], [863, 1525, 48, 822, 10, 149, 12, 9268, 14352, 45, 8947, 3, 23, 11410, 58, 1], [863, 1525, 48, 822, 10, 363, 19, 8, 2530, 13, 280, 58, 1], [863, 1525, 48, 822, 10, 149, 103, 8, 11110, 30, 12541, 161, 58, 1], [863, 1525, 48, 822, 10, 3, 30010, 16, 710, 936, 549, 7645, 3, 184, 39

In [25]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"],
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [26]:
# Map the preprocessing function across our dataset
tokenized_dataset = yahoo_answers_qa.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [51]:
tokenized_dataset["train"][2]

{'id': '2946427',
 'question': 'how to become a don ?',
 'answer': 'first you must prove your loyalty, be a "soldier", become a made man and Never be a rat.  watch the godfather movies',
 'nbestanswers': ['first you must prove your loyalty, be a "soldier", become a made man and Never be a rat.  watch the godfather movies',
  'enter into politics . u will become don . in very short time andwud even enjoy vip status . heheheh',
  'Begin with drugs.',
  'Change your name to Donald',
  'I don have any idea.',
  'you need lots of guts to become a don.Leave it man you will die in an encounter.',
  'Money and the mind 2 do it!!!!!!!!!!!!!!'],
 'main_category': 'Politics & Government',
 'input_ids': [863, 1525, 48, 822, 10, 149, 12, 582, 3, 9, 278, 3, 58, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [166,
  25,
  398,
  4410,
  39,
  13260,
  6,
  36,
  3,
  9,
  96,
  7,
  1490,
  972,
  1686,
  582,
  3,
  9,
  263,
  388,
  11,
  8400,
  36,
  3,
  9,
  3,
 

In [31]:
nltk.download("punkt", quiet=True)
nltk.download('punkt_tab')
metric = evaluate.load("rouge")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [34]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

   # Force the tokenizer to handle unknown tokens
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, errors='ignore') # or errors='replace'
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, errors='ignore') # or errors='replace'

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [35]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-35-9081caaebb41>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.763346,0.153951,0.024929,0.122804,0.137691
2,No log,3.719671,0.149662,0.024057,0.120265,0.132401
3,No log,3.702054,0.153296,0.025069,0.124007,0.136543


TrainOutput(global_step=375, training_loss=2.9048785807291666, metrics={'train_runtime': 384.5362, 'train_samples_per_second': 7.802, 'train_steps_per_second': 0.975, 'total_flos': 30468615536640.0, 'train_loss': 2.9048785807291666, 'epoch': 3.0})

In [36]:
last_checkpoint = "./results/checkpoint-375"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

In [48]:
my_question = "how to delete  pics from apple ipod?"
inputs = "Please answer to this question: " + my_question
input_dict = tokenizer(inputs, return_tensors="pt")  # Create a dictionary containing the input_ids
outputs = finetuned_model.generate(**input_dict)  # Pass the dictionary to generate
answer = tokenizer.decode(outputs[0])
from textwrap import fill

print(fill(answer, width=80))

<pad>Delete the pics from your iPod.. . Delete the pictures from your iPod..
